# Video Generation and Comparisons

This notebook demonstrates how to create various types of videos and animations to showcase the diamond segmentation pipeline.

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

from src.data.loader import DiamondDataLoader, DiamondShapeMapper
from src.pipe.video_comparison import (
    VideoComparator,
    create_triple_split_video,
    create_comparison_grid,
    create_annotated_comparison
)
from src.pipe.animation import (
    MaskEvolutionAnimator,
    AlgorithmCoverageAnimator,
    create_pipeline_animation
)
from src.utils.file_utils import ensure_dir

%matplotlib inline

## 1. Setup and Configuration

In [ ]:
# Configure paths
DATA_PATH = '../data/raw'
OUTPUT_DIR = '../videos'
VARIANT_1D = 'Shape_1d_256i'
VARIANT_5D = 'Shape_5d_256i'

# Create output directory
ensure_dir(OUTPUT_DIR)

# Initialize loaders
loader_1d = DiamondDataLoader(DATA_PATH, VARIANT_1D)
loader_5d = DiamondDataLoader(DATA_PATH, VARIANT_5D)
mapper = DiamondShapeMapper()

# Video settings
FPS = 15
ITERATIONS = 5

print(f"Output directory: {OUTPUT_DIR}")
print(f"Video FPS: {FPS}")
print(f"GrabCut iterations: {ITERATIONS}")

## 2. Triple-Split Video (Before | CLAHE | After)

Demonstrates the complete segmentation pipeline for a single diamond variation.

In [ ]:
# Select shape for demo
DEMO_SHAPE = 'AS'  # Asscher
shape_name = mapper.get_shape_name(DEMO_SHAPE)
shape_id = mapper.get_shape_id(DEMO_SHAPE)

print(f"Creating triple-split video for: {shape_name} ({DEMO_SHAPE})")

# Get image list from 1d dataset
image_list = loader_1d.list_images(shape_id)
print(f"Found {len(image_list)} images")

# Get full paths
image_paths = [loader_1d.get_image_path(shape_id, img) for img in image_list]

# Create video
output_path = Path(OUTPUT_DIR) / f'triple_split_{DEMO_SHAPE}_{shape_name}.avi'
comparator = VideoComparator(output_path, fps=FPS)

video_path = comparator.create_triple_split(
    image_paths,
    iterations=ITERATIONS,
    add_labels=True
)

print(f"\n✓ Video created: {video_path}")

## 3. Five-Split Video (5 Variations)

Shows segmentation results for 5 different variations of the same diamond shape side-by-side.

In [ ]:
from src.utils.segmentation import preprocess_for_segmentation, remove_background
from tqdm import tqdm

print(f"Creating five-split video for: {shape_name} ({DEMO_SHAPE})")

# Get all images from 5d dataset
all_images_5d = loader_5d.list_images(shape_id)
print(f"Found {len(all_images_5d)} total images in 5d dataset")

# Split into 5 variations (256 images each)
images_per_variation = 256
variations = []

for var_idx in range(5):
    start_idx = var_idx * images_per_variation
    end_idx = start_idx + images_per_variation
    var_images = all_images_5d[start_idx:end_idx]
    variations.append(var_images)

print(f"Processing {len(variations)} variations with {images_per_variation} images each")

# Process all variations
all_frames = [[] for _ in range(5)]

for var_idx in range(5):
    print(f"\nProcessing variation {var_idx + 1}/5...")
    
    for img_name in tqdm(variations[var_idx][:50], desc=f"Variation {var_idx+1}"):  # Limit to 50 frames for demo
        img_path = loader_5d.get_image_path(shape_id, img_name)
        original = cv2.imread(img_path)
        
        if original is None:
            continue
        
        # Process
        _, enhanced = preprocess_for_segmentation(original)
        segmented, _ = remove_background(original, enhanced, ITERATIONS)
        
        all_frames[var_idx].append(segmented)

# Create labels
labels = [f"{shape_name} Var {i+1}" for i in range(5)]

# Generate grid video
output_path = Path(OUTPUT_DIR) / f'five_split_{DEMO_SHAPE}_{shape_name}.avi'

video_path = create_comparison_grid(
    all_frames,
    labels,
    output_path,
    fps=FPS,
    grid_layout=(1, 5)
)

print(f"\n✓ Five-split video created: {video_path}")

## 4. Mask Evolution Animation

Visualize how the GrabCut mask evolves over iterations for a single image.

In [ ]:
# Select a single image for animation
sample_image = image_paths[0]

print(f"Creating mask evolution animation...")
print(f"Sample image: {sample_image}")

# Create animation
output_path = Path(OUTPUT_DIR) / f'mask_evolution_{DEMO_SHAPE}.avi'
animator = MaskEvolutionAnimator(output_path, fps=10)

video_path = animator.animate_single_image_evolution(
    sample_image,
    max_iterations=10,
    frames_per_iteration=5
)

print(f"\n✓ Mask evolution animation created: {video_path}")

## 5. Pipeline Steps Animation

Step-by-step visualization of the entire segmentation pipeline.

In [ ]:
print(f"Creating pipeline steps animation...")

output_path = Path(OUTPUT_DIR) / f'pipeline_steps_{DEMO_SHAPE}.avi'

video_path = create_pipeline_animation(
    sample_image,
    output_path,
    iterations=ITERATIONS,
    fps=10
)

print(f"\n✓ Pipeline animation created: {video_path}")

## 6. Annotated Comparison with Progress Bar

Create a professional video with progress indicators and frame counters.

In [ ]:
print(f"Creating annotated comparison video...")

# Use subset of images for demo
demo_images = image_paths[:100]  # First 100 images

output_path = Path(OUTPUT_DIR) / f'annotated_{DEMO_SHAPE}_{shape_name}.avi'

video_path = create_annotated_comparison(
    demo_images,
    output_path,
    iterations=ITERATIONS,
    fps=FPS,
    show_progress=True,
    show_counter=True
)

print(f"\n✓ Annotated video created: {video_path}")

## 7. Multi-Shape Comparison

Create a comparison video showing segmentation across different diamond shapes.

In [ ]:
# Select multiple shapes for comparison
comparison_shapes = ['AS', 'BR', 'EM']

print(f"Creating multi-shape comparison video...")
print(f"Shapes: {', '.join(comparison_shapes)}")

# Process first 50 images from each shape
all_shape_frames = []
shape_labels = []

for shape_code in comparison_shapes:
    shape_id = mapper.get_shape_id(shape_code)
    shape_name = mapper.get_shape_name(shape_code)
    
    print(f"\nProcessing {shape_name} ({shape_code})...")
    
    images = loader_1d.list_images(shape_id)[:50]  # First 50 images
    frames = []
    
    for img_name in tqdm(images, desc=shape_code):
        img_path = loader_1d.get_image_path(shape_id, img_name)
        original = cv2.imread(img_path)
        
        if original is None:
            continue
        
        _, enhanced = preprocess_for_segmentation(original)
        segmented, _ = remove_background(original, enhanced, ITERATIONS)
        
        frames.append(segmented)
    
    all_shape_frames.append(frames)
    shape_labels.append(f"{shape_name}")

# Create comparison video
output_path = Path(OUTPUT_DIR) / 'multi_shape_comparison.avi'

video_path = create_comparison_grid(
    all_shape_frames,
    shape_labels,
    output_path,
    fps=FPS,
    grid_layout=(1, 3)
)

print(f"\n✓ Multi-shape comparison video created: {video_path}")

## 8. Video Information Summary

In [ ]:
from src.pipe.video_creator import get_video_info
import os

print("="*60)
print("GENERATED VIDEOS SUMMARY")
print("="*60)

video_files = glob.glob(str(Path(OUTPUT_DIR) / '*.avi'))

for video_file in sorted(video_files):
    try:
        info = get_video_info(video_file)
        file_size = os.path.getsize(video_file) / (1024 * 1024)  # MB
        
        print(f"\n{os.path.basename(video_file)}")
        print(f"  Resolution: {info['width']}x{info['height']}")
        print(f"  FPS: {info['fps']:.2f}")
        print(f"  Frames: {info['frame_count']}")
        print(f"  Duration: {info['duration_seconds']:.2f}s")
        print(f"  File size: {file_size:.2f} MB")
    except Exception as e:
        print(f"\n{os.path.basename(video_file)}: Error reading info - {e}")

print("\n" + "="*60)
print(f"Total videos: {len(video_files)}")
print("="*60)

## Summary

This notebook demonstrated:

1. **Triple-Split Videos**: Showing Before | CLAHE | After pipeline stages
2. **Five-Split Videos**: Comparing 5 variations of the same diamond shape
3. **Mask Evolution**: Visualizing how GrabCut refines the segmentation
4. **Pipeline Animation**: Step-by-step breakdown of the algorithm
5. **Annotated Videos**: Professional outputs with progress bars and counters
6. **Multi-Shape Comparison**: Side-by-side comparison across shapes

All videos are saved in the `videos/` directory and can be used for presentations, documentation, or further analysis.